In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO


In [ ]:
THRESHOLD_DISTANCE = 100

def calculate_distance(p1, p2):
    """Compute Euclidean distance between two points."""
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)


In [ ]:
video_path = r"C:\WoSAAP\Voilence Against Woman DATASET\WhatsApp Video 2025-02-16 at 18.10.48_0a71d212.mp4"
# Load YOLOv8 Model
model = YOLO("yolov8n.pt")

cap = cv2.VideoCapture(video_path)   
assert cap.isOpened(), "Error: Could not open video file"

In [ ]:

cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Stop if video ends

    # Run YOLOv8 for Person Detection
    results = model(frame)

    # Store detected people positions
    people_positions = []

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())  # Convert tensor to list and map to int
            class_id = int(box.cls[0])  # Object class ID

            if class_id == 0:  # YOLO class ID 0 corresponds to "person"
                # Calculate the center of the bounding box
                center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
                people_positions.append(((center_x, center_y), (x1, y1, x2, y2)))

    # Detect Lone Persons
    lone_persons = []
    for person in people_positions:
        center, bbox = person
        is_lone = True  # Assume person is lone unless proven otherwise

        for other_person in people_positions:
            if person == other_person:
                continue  # Skip self comparison

            other_center, _ = other_person
            distance = calculate_distance(center, other_center)

            if distance < THRESHOLD_DISTANCE:
                is_lone = False  # If another person is close, this person is not lone
                break

        if is_lone:
            lone_persons.append(bbox)

    # Draw Bounding Boxes and Labels for All Detected Persons
    for person in people_positions:
        (center_x, center_y), (x1, y1, x2, y2) = person  # Unpack the tuple correctly
        color = (0, 255, 0)  # Green for normal detection
        label = "Person"

        # If person is lone, change bounding box color
        if (x1, y1, x2, y2) in lone_persons:
            color = (0, 0, 255)  # Red for lone persons
            label = "Lone Person"

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Display the Processed Video Frame
    cv2.imshow("Lone Person Detection", frame)
    
    # Press 'q' to Quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
